In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [4]:
def convert_y(dataY):
    y = []
    for i in range (len(dataY)):
        if(dataY[i] > 50):
            y.append(0)
        else:
            y.append(1)
    return np.array(y)

In [5]:
dataX = pd.read_csv('./datasets/spotofy_music_normalized.csv')
dataY = pd.read_csv('./datasets/spotofy_music_labels.csv')
dataY_multi = pd.read_csv('./datasets/spotofy_music_labels.csv')
Y = convert_y(np.array(dataY))
dataX.head()

,Explicit,Artist_followers,Track_number_on_album,Acousticness,Danceability,Energy,Liveness,Loudness,Speechiness,Tempo,Mode,Key,Valence
0,1.112882,1.012971,-0.703623,-0.253508,-0.056900,-0.257351,-0.321798,-0.287111,1.295366,2.132090,0.865448,0.478965,0.209096
1,-0.898356,0.172558,-0.703623,0.798171,0.424214,-0.688588,-0.492682,0.282796,-0.811102,-0.910119,0.865448,0.752338,0.774957
2,-0.898356,-0.658039,-0.703623,0.764382,0.223151,0.783781,0.019972,0.893382,-0.654093,-0.677720,-1.155199,1.025710,1.138724
3,-0.898356,-0.471741,1.732316,-0.619272,-1.528965,0.506556,-0.410955,0.398452,-0.609112,2.593547,0.865448,0.752338,-1.555849
4,-0.898356,0.029319,-0.703623,-0.683893,0.330863,-1.138308,1.097289,-0.823916,-0.601473,-0.142817,0.865448,-1.434643,-1.806445


We will implement ANNs on the above data and see the accuracy scores of its prediction of hit vs not hit.

In [6]:
# import keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [7]:
#Normalizing the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(dataX)

In [14]:
def create_baseline():
	# create model
	model = Sequential()
	model.add(Dense(13, input_dim=13, activation='relu'))
	model.add(Dense(1, activation='sigmoid')) #Single Layer Sigmoid Binary Classifier == Logistic Regression
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [26]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=25, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=5, shuffle=True)
results = cross_val_score(estimator, dataX, Y, cv=kfold)
print("Baseline with Un-Standardized Data, KFold with K = 5: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
results2 = cross_val_score(estimator, X, Y, cv=kfold)
print("Baseline with Standardized Data, KFold with K = 5: %.2f%% (%.2f%%)" % (results2.mean()*100, results2.std()*100))

<ipython-input-26-92b082f334d5>:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  estimator = KerasClassifier(build_fn=create_baseline, epochs=25, batch_size=5, verbose=0)


Baseline with Un-Standardized Data, KFold with K = 5: 68.66% (1.39%)
Baseline with Standardized Data, KFold with K = 5: 68.94% (1.08%)


In [8]:
def create_neural_network(activation = "tanh", neurons = 5):
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, activation='relu'))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(1, activation='sigmoid')) #Single Layer Sigmoid Binary Classifier == Logistic Regression
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [10]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_neural_network, epochs=40, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=3, shuffle=True)
results = cross_val_score(pipeline, dataX, Y, cv=kfold)
print("NN Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

C:\Users\Tushar\AppData\Local\Temp/ipykernel_4944/3792645775.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  estimators.append(('mlp', KerasClassifier(build_fn=create_neural_network, epochs=40, batch_size=5, verbose=1)))


Epoch 1/40
567/567 [==============================] - 2s 2ms/step - loss: 0.6425 - accuracy: 0.6552
Epoch 2/40
567/567 [==============================] - 1s 2ms/step - loss: 0.6016 - accuracy: 0.6902
Epoch 3/40
567/567 [==============================] - 1s 2ms/step - loss: 0.5932 - accuracy: 0.6969
Epoch 4/40
567/567 [==============================] - 1s 2ms/step - loss: 0.5889 - accuracy: 0.7033
Epoch 5/40
567/567 [==============================] - 1s 2ms/step - loss: 0.5853 - accuracy: 0.7058
Epoch 6/40
567/567 [==============================] - 1s 2ms/step - loss: 0.5826 - accuracy: 0.7043
Epoch 7/40
567/567 [==============================] - 1s 2ms/step - loss: 0.5795 - accuracy: 0.7065
Epoch 8/40
567/567 [==============================] - 1s 2ms/step - loss: 0.5770 - accuracy: 0.7111
Epoch 9/40
567/567 [==============================] - 1s 2ms/step - loss: 0.5751 - accuracy: 0.7128
Epoch 10/40
567/567 [==============================] - 1s 2ms/step - loss: 0.5735 - accuracy: 0.7121

With the ReLU activation function on running for 25 epochs, the K-Fold CV accuracy on K=3 comes out to be 73.5% and the accuracy on K=5 is 71.5%. The Neural Network model trained was a simple one with one hidden layer with 5 neurons.

With the tanh activation function on running for 40 epochs, the K-Fold CV accuracy on K=3 comes out to be 66.89%. The Neural Network model trained was a simple one with one hidden layer with 10 neurons.
and the accuracy on K=5 is 67.93% --> hiddlen layer with 5 neurons

> Accuracy: 66.89% (1.32%)

>Accuracy: 67.93% (0.89%)

>Accuracy: 69.20% (0.90%) --> leaky_relu, 5 neurons, k=5

>NN Accuracy: 68.75% (0.31%) --> leaky_relu, 5 neurons, k=3

>NN Accuracy: 71.3% (1.16%) --> tanh , 5 neurons, k=5, epcohs 25

> Accuracy : 71.8% --> tanh, 5 neurons, standarized, 40 epoch, k=3

In [32]:
results

array([0.67647058, 0.68470585, 0.67491168, 0.6949352 , 0.65135455])